In [ ]:
# Migrated from https://github.com/RylanSchaeffer/KoyejoLab-Revisiting-Model-Collapse/blob/main/language_modeling_experiments/plot_matthias_cameraready.ipynb


wandb_entity = "mgerstgrasser"
group_name = "tinystories_models_1"

run_groups_to_fetch = [
    "tinystories_models_1",
    # "tinystories_1_matthias_8k_1epoch_llama",
    # "tinystories_1_matthias_8k_1epoch_llama42M_snap",
    # "tinystories_1_matthias_8k_1epoch_llama125M_nlp",
    # "tinystories_5_matthias_8k_3epoch_new",
    # "tinystories_5_matthias_8k_1epoch_new",
    # "tinystories_1_matthias_8k_1epoch_subsample",
    # "tinystories_llama_firstiter",
]

In [ ]:
from datetime import datetime
import joblib
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import wandb


import src.plot

In [ ]:
# We have the same vocab size everywhere, so same uniform loss.
uniform_loss = math.log(8000)

In [ ]:
# Create API to retrieve the runs from wandb
api = wandb.Api(timeout=100)

In [ ]:
# Get the current date and time
# query_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
# query_datetime = "20240327_185710"
# query_datetime = "20240329_220900"
query_datetime = "20240809"
plot_dir = "figures"
data_dir = "data"
os.makedirs(plot_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)


def load_data(group_name, query_datetime, data_dir: str = "data"):
    print(f"Loading data for group {group_name} at t= {query_datetime}.")
    os.makedirs(data_dir, exist_ok=True)
    cached_results_path = os.path.join(
        data_dir, f"cache_{group_name}_{query_datetime}.joblib"
    )
    if os.path.exists(cached_results_path):
        cached_results = joblib.load(cached_results_path)
        # Load data from cache if the file exists
        data_inner = cached_results["df_inner"]
        data_outer = cached_results["df_outer"]
        print(f"Loaded data_inner and data_outer for group {group_name} from disk.")
    else:
        print(
            f"Data for group {group_name} at t= {query_datetime} not found. Fetching data from W&B API."
        )

        # Your existing code to fetch data from the API
        runs = api.runs(
            f"{wandb_entity}/model_collapse_gpt",
            filters={
                "group": group_name,
            },
        )

        data_inner = []
        data_outer = []
        model_num_parameters = None

        # Iterate over the runs and extract the required data
        for run in runs:
            if "dataset_mode" in run.config and "model_num_parameters" in run.config:
                dataset_mode = run.config["dataset_mode"]
                model_num_parameters = run.config["model_num_parameters"]
                model = run.config["model"]
                history = run.scan_history(page_size=100000)
                run_config = {
                    "temperature": run.config["full_cli_args"][
                        "generation_temperature"
                    ],
                    "model": model,
                    "model_num_parameters": model_num_parameters,
                    "dataset_mode": dataset_mode,
                    "run_group": group_name,
                    "run_id": run.id,
                    "dataset_fraction": (
                        run.config["dataset_fraction"]
                        if run.config["dataset_fraction"] is not None
                        else 1.0
                    ),
                    "generation_strategy": (
                        run.config["full_cli_args"]["generation_strategy"]
                    ),
                    "num_epochs": run.config["epochs"],
                    "firstiter_steps": run.config["full_cli_args"].get(
                        "num_steps_first_iter", -1
                    ),
                    "hyperparameters": f"{model}_{model_num_parameters}_frac{run.config['dataset_fraction'] if run.config['dataset_fraction'] is not None else 1.0}_epochs{run.config['epochs']}_temp{run.config['full_cli_args']['generation_temperature']}",
                }
                for row in history:
                    try:
                        if row["is_outer"] is True:
                            data_outer.append(
                                {
                                    "step": row["_step"],
                                    "iteration": row.get("iteration", np.nan),
                                    "trainer_global_step": row.get(
                                        "trainer_global_step", np.nan
                                    ),
                                    "eval_loss_outer": row.get(
                                        "eval_loss_outer", np.nan
                                    ),
                                    "eval_perplexity_outer": row.get(
                                        "eval_perplexity_outer", np.nan
                                    ),
                                    **run_config,
                                }
                            )
                        else:
                            data_inner.append(
                                {
                                    "step": row["_step"],
                                    "loss": row.get("loss", np.nan),
                                    "eval_loss": row.get("eval_loss", np.nan),
                                    "iteration": row.get("iteration", np.nan),
                                    "trainer_global_step": row.get(
                                        "trainer_global_step", np.nan
                                    ),
                                    "epoch": row["epoch"],
                                    **run_config,
                                }
                            )
                    except KeyError as ke:
                        print(f"KeyError {ke} in run {run.id}, row {row}.")

        # df_inner = pd.DataFrame(data_inner)
        # df_outer = pd.DataFrame(data_outer)

        # Save the DataFrames to cache as CSV files
        cached_results = {
            "df_inner": data_inner,
            "df_outer": data_outer,
        }
        joblib.dump(cached_results, filename=cached_results_path)

        print(
            f"Data for group {group_name} at t=  {query_datetime} fetched from W&B API and written to disk."
        )

    return data_inner, data_outer


# Load or fetch the data
data_inner = []
data_outer = []
for group_name in run_groups_to_fetch:
    data_inner_group, data_outer_group = load_data(
        group_name, query_datetime, data_dir=data_dir
    )
    data_inner.extend(data_inner_group)
    data_outer.extend(data_outer_group)

df_inner = pd.DataFrame(data_inner)
df_outer = pd.DataFrame(data_outer)


# Skip this, not sure why I even ran this.
df_inner = df_inner[df_inner["hyperparameters"] != "gpt2_8890880_frac5_epochs3_temp0.3"]
df_outer = df_outer[df_outer["hyperparameters"] != "gpt2_8890880_frac5_epochs3_temp0.3"]

# Skip subsample run with wrong batch size
df_inner = df_inner[df_inner["run_id"] != "dty86yke"]
df_outer = df_outer[df_outer["run_id"] != "dty86yke"]
df_inner = df_inner[df_inner["run_id"] != "3ov1gya8"]
df_outer = df_outer[df_outer["run_id"] != "3ov1gya8"]


df_inner["dataset_mode"] = df_inner["dataset_mode"].map(
    {
        "concatenate": "Accumulate",
        "replace": "Replace",
        "replace_multiple": "Replace (Multiple)",
        "concatenate_subsample": "Accumulate-Subsample",
    }
)

df_outer["dataset_mode"] = df_outer["dataset_mode"].map(
    {
        "concatenate": "Accumulate",
        "replace": "Replace",
        "replace_multiple": "Replace (Multiple)",
        "concatenate_subsample": "Accumulate-Subsample",
    }
)

# def construct_model_name(row):
#     if row["model"] == "gpt2":
#         model_name_str = "GPT-2-8M"
#     elif row["run_group"] == "tinystories_1_matthias_8k_1epoch_llama42M_snap":
#         model_name_str = "Llama-2-42M"
#     elif row["run_group"] == "tinystories_1_matthias_8k_1epoch_llama125M_nlp":
#         model_name_str = "Llama-2-125M"
#     elif row["run_group"] == "tinystories_1_matthias_8k_1epoch_llama":
#         model_name_str = "Llama-2-12M"
#     else:
#         raise ValueError(f"Unknown model: {row['model']}")
#     return f"{model_name_str} ({row['model_num_parameters']/1e6:.0f}M)"


def construct_model_name(row):
    if row["model"] == "gpt2":
        return "GPT-2 (9M)"
    elif row["model"] == "llama2":
        return "Llama-2 (12M)"
    elif row["model"] == "llama2-42M":
        return "Llama-2 (42M)"
    elif row["model"] == "llama2-125M":
        return "Llama-2 (126M)"
    else:
        raise ValueError(f"Unknown model: {row['model']}")
    return f"{model_name_str} ({row['model_num_parameters']/1e6:.0f}M)"


df_inner["Model"] = df_inner.apply(lambda row: construct_model_name(row=row), axis=1)
df_outer["Model"] = df_outer.apply(lambda row: construct_model_name(row=row), axis=1)

# hyperparams_to_nice_model_name = {
#     'gpt2_8890880_frac1.0_epochs1_temp0.3': 'GPT-2-9M (temp=0.3)',
#     'gpt2_8890880_frac1.0_epochs1_temp1': 'GPT-2-9M',
#     'llama2_12488960_frac1.0_epochs1_temp1': 'Llama-2-12M',
#     'llama2_41755136_frac1.0_epochs1_temp1': 'Llama-2-42M',
#     'llama2_125553408_frac1.0_epochs1_temp1': 'Llama-2-125M',
#     'gpt2_8890880_frac5_epochs3_temp0.3': 'GPT-2-8M (temp=0.3, small dataset, 3 epochs)',
#     'gpt2_8890880_frac5_epochs3_temp1': 'GPT-2-8M (small dataset, 3 epochs)',
#     'gpt2_8890880_frac5_epochs1_temp1': 'GPT-2-8M (small dataset)'
# }

hyperparams_to_nice_model_name = {
    "gpt2_8890880_frac1.0_epochs1_temp0.3": " (temp=0.3)",
    "gpt2_8890880_frac1.0_epochs1_temp1": "",
    "llama2_12488960_frac1.0_epochs1_temp1": "",
    "llama2_41755136_frac1.0_epochs1_temp1": "",
    "llama2_125553408_frac1.0_epochs1_temp1": "",
    "gpt2_8890880_frac5_epochs3_temp0.3": " (temp=0.3, small dataset, 3 epochs)",
    "gpt2_8890880_frac5_epochs3_temp1": " (small dataset, 3 epochs)",
    "gpt2_8890880_frac5_epochs1_temp1": " (small dataset)",
}


def construct_model_name_long(row):
    return row["Model"]
    if (
        row["hyperparameters"].startswith("gpt2_8890880_frac1.0_epochs1_temp")
        and not row["hyperparameters"] in hyperparams_to_nice_model_name
    ):
        model_name_str = row["Model"] + "(" + str(row["temperature"]) + ")"
    elif row["hyperparameters"] in hyperparams_to_nice_model_name:
        model_name_str = (
            row["Model"] + hyperparams_to_nice_model_name[row["hyperparameters"]]
        )
    else:
        raise ValueError(f"Unknown model: {row['model']}")
    return model_name_str


df_inner["Model (long)"] = df_inner.apply(
    lambda row: construct_model_name_long(row=row), axis=1
)
df_outer["Model (long)"] = df_outer.apply(
    lambda row: construct_model_name_long(row=row), axis=1
)

In [ ]:
df_inner["run_id"].unique()

In [ ]:
df_inner["Model (long)"].unique()

In [ ]:
df_inner.head()

In [ ]:
col_order = ["Replace", "Accumulate-Subsample", "Accumulate"]

In [ ]:
# Filter the DataFrame for trainer global step vs eval loss plot
df_trainer_eval_loss = df_inner[
    [
        "trainer_global_step",
        "eval_loss",
        "loss",
        "iteration",
        "dataset_mode",
        "epoch",
        "temperature",
        "model",
        "model_num_parameters",
        "run_group",
        "Model",
        "Model (long)",
        "generation_strategy",
    ]
].dropna()

df_trainer_eval_loss = df_trainer_eval_loss[df_trainer_eval_loss["temperature"] == 1.0]

df_trainer_eval_loss = df_trainer_eval_loss[
    df_trainer_eval_loss["generation_strategy"] == "first_token_inclusive"
]

df_trainer_eval_loss = df_trainer_eval_loss[df_trainer_eval_loss["iteration"] < 5]


# We use 1-based indexing in the mathematical notation.
df_trainer_eval_loss["iteration"] += 1


# Rename column from "iteration" to "Iteration" for nicer plotting.
df_trainer_eval_loss = df_trainer_eval_loss.rename(
    columns={"iteration": "Iteration", "temperature": "Temperature"}
)

In [ ]:
df_fig_1 = df_trainer_eval_loss
df_fig_1 = df_fig_1.rename(columns={"Model (long)": "Model", "Model": "Model (short)"})
df_fig_1 = df_fig_1[df_fig_1["Model"] != "Llama-2-12M"]
# Create a relplot for trainer global step vs eval loss
g = sns.relplot(
    # data=df_trainer_eval_loss[df_trainer_eval_loss["Temperature"] == 0.3],
    data=df_fig_1[df_fig_1["Temperature"] == 1.0],
    x="epoch",
    y="eval_loss",
    hue="Iteration",
    # style="Temperature",
    # style_order=[1.0, 0.3],
    row="dataset_mode",
    col="Model",
    row_order=col_order,
    kind="line",
    # palette="mako",
    palette="copper",
    # palette="crest",
    # palette="cool",
    # palette="flare",
    facet_kws={"sharey": True, "sharex": True, "margin_titles": True},
    legend="full",
)
g.set_axis_labels("Epoch", "Cross Entropy (Test)")
g.set_titles(
    col_template="{col_name}",
    row_template="{row_name}",
)
# g.map(
#     plt.axhline, y=uniform_loss, color="red", linestyle="--", label="Uniform"
# )  # Add horizontal line
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
g.fig.suptitle(f"Language Models Pretrained on TinyStories")
src.plot.save_plot_with_multiple_extensions(
    plot_dir=plot_dir,
    plot_title=f"eval_loss_linear_vs_epoch_linear_col_iteration_{query_datetime}",
)

plt.show()

In [ ]:
df_fig_1 = df_trainer_eval_loss
df_fig_1 = df_fig_1.rename(columns={"Model (long)": "Model", "Model": "Model (short)"})
df_fig_1 = df_fig_1[df_fig_1["Model"] != "Llama-2-12M"]

for xaxis in ["trainer_global_step", "epoch"]:
    for scale in ["linear", "log"]:
        # Create a relplot for trainer global step vs eval loss
        g = sns.relplot(
            # data=df_trainer_eval_loss[df_trainer_eval_loss["Temperature"] == 0.3],
            data=df_fig_1[df_fig_1["Temperature"] == 1.0],
            x=xaxis,
            y="eval_loss",
            hue="Iteration",
            # style="Temperature",
            # style_order=[1.0, 0.3],
            col="dataset_mode",
            row="Model",
            col_order=col_order,
            kind="line",
            # palette="mako",
            palette="copper",
            # palette="crest",
            # palette="cool",
            # palette="flare",
            facet_kws={"sharey": True, "sharex": True, "margin_titles": True},
            legend="full",
        )
        if scale == "log":
            g.set(xscale="log", yscale="log")
        if xaxis == "epoch":
            g.set_axis_labels("Epoch", "Cross Entropy (Test)")
        else:
            g.set_axis_labels("Gradient Step", "Cross Entropy (Test)")
        g.set_titles(
            col_template="{col_name}",
            row_template="{row_name}",
        )
        # g.map(
        #     plt.axhline, y=uniform_loss, color="red", linestyle="--", label="Uniform"
        # )  # Add horizontal line
        sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
        g.fig.suptitle(f"Language Models Pretrained on TinyStories")
        src.plot.save_plot_with_multiple_extensions(
            plot_dir=plot_dir,
            plot_title=f"eval_loss_{scale}_vs_{xaxis}_linear_flipped_{query_datetime}",
        )

        plt.show()

In [ ]:
cond = ((df_inner["epoch"] == 1.0) & (df_inner["num_epochs"] == 1.0)) | (
    (df_inner["epoch"] == 3.0) & (df_inner["num_epochs"] == 3.0)
)
# df_fig1_alt = df_inner[cond]
cond = df_trainer_eval_loss["epoch"] == 1.0
df_fig1_alt = df_trainer_eval_loss[cond]

In [ ]:
df_fig1_alt = df_fig1_alt
# df_fig1_alt = df_fig1_alt.rename(columns={"Model (long)": "Model", "Model": "Model (short)"})
# df_fig1_alt = df_fig1_alt[df_fig1_alt["Model"] != "Llama-2-12M"]

# Create a relplot for trainer global step vs eval loss
g = sns.relplot(
    # data=df_trainer_eval_loss[df_trainer_eval_loss["Temperature"] == 0.3],
    data=df_fig1_alt[df_fig1_alt["Temperature"] == 1.0]
    .rename(columns={"Model": "Model (short)"})
    .rename(
        columns={
            "Model (long)": "Model",
        }
    ),
    x="Iteration",
    y="eval_loss",
    # hue="Iteration",
    # style="Temperature",
    # style_order=[1.0, 0.3],
    col="dataset_mode",
    hue="Model",
    hue_order=[
        "GPT-2 (9M)",
        "Llama-2 (12M)",
        "Llama-2 (42M)",
        "Llama-2 (126M)",
    ],
    col_order=col_order,
    kind="line",
    # palette="mako",
    # palette="copper",
    # palette="crest",
    # palette="cool",
    # palette="flare",
    facet_kws={"sharey": True, "sharex": True, "margin_titles": True},
    legend="full",
    marker="o",
    markersize=10,
)
g.set_axis_labels("Model-Fitting Iteration", "Cross Entropy (Test)")
g.set_titles(
    col_template="{col_name}",
    row_template="{row_name}",
)
# g.set( yscale="log")
# g.map(
#     plt.axhline, y=uniform_loss, color="red", linestyle="--", label="Uniform"
# )  # Add horizontal line
# sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
g.fig.suptitle(f"Language Models Pretrained on TinyStories")
src.plot.save_plot_with_multiple_extensions(
    plot_dir=plot_dir,
    plot_title=f"eval_loss_linear_vs_iteration_{query_datetime}",
)

plt.show()